In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
%matplotlib inline
import plotly
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import folium

import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [2]:
# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [3]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [4]:

#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(100)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,823786,44845,667592,32930,156194,11915,5.4,North America
1,Spain,204178,21282,184948,19315,19230,1967,10.4,Europe
2,Italy,183957,24648,168941,22170,15016,2478,13.4,Europe
3,France,159297,20829,147091,17941,12206,2888,13.1,Europe
4,Germany,148291,5033,137698,4052,10593,981,3.4,Europe
...,...,...,...,...,...,...,...,...,...
95,Bolivia,598,34,441,29,157,5,5.7,South America
96,Kyrgyzstan,590,7,466,5,124,2,1.2,NaN
97,Uruguay,535,11,502,9,33,2,2.1,South America
98,Kosovo,510,12,449,11,61,1,2.4,Europe


In [5]:
#import requests
#import json
#import os


# fetch data data and make dataframe

# latest data for all country

#all_data_url = "https://lab.isaaclin.cn/nCoV/api/area?latest=1"
#all_data = requests.get(all_data_url).json()  # get data in json format

#all_data_df = pd.DataFrame(all_data)

#all_data_df = pd.json_normalize(all_data_df['results'])

#filter_all_data = all_data_df.filter(items=['continentEnglishName', 'countryEnglishName',
#                                            'currentConfirmedCount', 'confirmedCount', 'curedCount', 'deadCount'])
#print("Latest update of all the country`s data\n")
#print(filter_all_data)

# save all data to a csv file called all_data.csv
#filter_all_data.to_csv('all_data.csv')
#print("Data saved to all_data.csv file")

In [6]:
# latest data for Bangladesh

# get the latest data for bangladesh
# fetch data set from this api
#url = "https://lab.isaaclin.cn/nCoV/api/area?latest=1&provinceEng=Bangladesh&lang=en"
#returned_data = requests.get(url).json()  # convert data to json
# to view returned data in a more presentable view
# content = json.dumps(returned_data, indent=4, sort_keys=True)
#df = pd.json_normalize(returned_data['results'])  # make a dataframe
# filtered_dataframe
#filtered_df = df.filter(items=['continentEnglishName', 'countryEnglishName',
#                               'currentConfirmedCount', 'confirmedCount', 'curedCount', 'deadCount', ])
#print("Latest update of Bangladesh`s data\n")
#print(filtered_df)

# save all data to a csv file called all_data.csv
#filtered_df.to_csv('bangladesh_data.csv')
#print("All data saved to bangladesh_data.csv file")

In [9]:
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,52,58,49,70,56,66,27,63,30,66
Albania,0,0,0,0,0,0,0,0,0,0,...,13,21,8,19,24,21,9,14,22,25
Algeria,0,0,0,0,0,0,0,0,0,0,...,89,69,87,90,108,150,116,95,89,93
Andorra,0,0,0,0,0,0,0,0,0,0,...,37,8,13,14,0,23,8,9,4,0
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0


In [22]:
#dataset of global cases
df_global_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", error_bad_lines = False, warn_bad_lines = True)
df_global_deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", error_bad_lines = False, warn_bad_lines = True)
df_global_recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv", error_bad_lines = False, warn_bad_lines = True)

In [23]:
#data preprocessing
df_global_cases.drop('Province/State', axis = 1, inplace = True)
df_global_cases.drop('Lat', axis = 1, inplace = True)
df_global_cases.drop('Long', axis = 1, inplace = True)
#df_global_cases.head()

In [24]:
df_global_deaths.drop('Province/State', axis = 1, inplace = True)
df_global_deaths.drop('Lat', axis = 1, inplace = True)
df_global_deaths.drop('Long', axis = 1, inplace = True)
#df_global_deaths.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,18,21,23,25,30,30,30,33,36,36
1,Albania,0,0,0,0,0,0,0,0,0,...,23,23,24,25,26,26,26,26,26,26
2,Algeria,0,0,0,0,0,0,0,0,0,...,293,313,326,336,348,364,367,375,384,392
3,Andorra,0,0,0,0,0,0,0,0,0,...,29,29,31,33,33,35,35,36,37,37
4,Angola,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [25]:
df_global_recovered.drop('Province/State', axis = 1, inplace = True)
df_global_recovered.drop('Lat', axis = 1, inplace = True)
df_global_recovered.drop('Long', axis = 1, inplace = True)
#df_global_recovered.head()

In [26]:
#function to melt the dataframe according to Country/Region
def melt_data(data_frame, case_type):
    melted_df = data_frame.melt(id_vars = ["Country/Region"])
    melted_df.rename(columns = {"variable" : "Date", "value" : case_type}, inplace = True)
    return melted_df

In [27]:
melted_df_global_cases = melt_data(df_global_cases, "Cases")
melted_df_global_cases = melted_df_global_cases.drop_duplicates()
#melted_df_global_cases.head()

In [28]:
melted_df_global_deaths = melt_data(df_global_deaths, "Deaths")
melted_df_global_deaths = melted_df_global_deaths.drop_duplicates()
#melted_df_global_deaths.head()

In [29]:
melted_df_global_recovered = melt_data(df_global_recovered, "Recovered")
melted_df_global_recovered = melted_df_global_recovered.drop_duplicates()
#melted_df_global_recovered.head()

In [30]:
#merging previous dataframes to a combined one
df_global_temp = pd.merge(melted_df_global_cases, melted_df_global_deaths, on = ['Country/Region', 'Date']) 
df_global_combined = pd.merge(df_global_temp, melted_df_global_recovered, on = ['Country/Region', 'Date'])
#df_global_combined.head()

In [31]:
#filtering data related to Bangladesh
df_bangladesh = df_global_combined.loc[df_global_combined["Country/Region"] == "Bangladesh"]
df_bangladesh['Date'] = pd.to_datetime(df_bangladesh['Date'])

In [32]:
bd_cases = df_bangladesh.groupby('Date').sum()['Cases'].reset_index()
bd_deaths = df_bangladesh.groupby('Date').sum()['Deaths'].reset_index()
bd_recovered = df_bangladesh.groupby('Date').sum()['Recovered'].reset_index()

In [44]:
#visualizing overall condition due to the coronavirus in Bangladesh
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x = bd_cases['Date'], y = bd_cases['Cases'], mode = 'lines+markers', name = 'Cases', line = dict(color = 'blue', width = 1)))
fig2.add_trace(go.Scatter(x = bd_deaths['Date'], y = bd_deaths['Deaths'], mode = 'lines+markers', name = 'Deaths', line = dict(color = 'red', width = 1)))
fig2.add_trace(go.Scatter(x = bd_recovered['Date'], y = bd_recovered['Recovered'], mode = 'lines+markers', name = 'Recovered', line = dict(color = 'green', width = 1)))

fig2.update_layout(title = "Analysis of Cases, Deaths, and Recoveries in Bangladesh", xaxis_title = "Date", yaxis_title = "Values", height = 600, width = 800)
   
fig2.show()

In [53]:
#filtering data related to USA
df_USA = df_global_combined.loc[df_global_combined["Country/Region"] == "China"]
df_USA['Date'] = pd.to_datetime(df_USA['Date'])

In [54]:
USA_cases = df_USA.groupby('Date').sum()['Cases'].reset_index()
USA_deaths = df_USA.groupby('Date').sum()['Deaths'].reset_index()
USA_recovered = df_USA.groupby('Date').sum()['Recovered'].reset_index()

In [38]:
#filtering data related to Spain
df_Spain = df_global_combined.loc[df_global_combined["Country/Region"] == "Spain"]
df_Spain['Date'] = pd.to_datetime(df_Spain['Date'])

In [39]:
Spain_cases = df_Spain.groupby('Date').sum()['Cases'].reset_index()
Spain_deaths = df_Spain.groupby('Date').sum()['Deaths'].reset_index()
Spain_recovered = df_Spain.groupby('Date').sum()['Recovered'].reset_index()

In [40]:
#filtering data related to Italy
df_Italy = df_global_combined.loc[df_global_combined["Country/Region"] == "Italy"]
df_Italy['Date'] = pd.to_datetime(df_Italy['Date'])

In [41]:
Italy_cases = df_Italy.groupby('Date').sum()['Cases'].reset_index()
Italy_deaths = df_Italy.groupby('Date').sum()['Deaths'].reset_index()
Italy_recovered = df_Italy.groupby('Date').sum()['Recovered'].reset_index()

In [56]:
from plotly.subplots import make_subplots

#subplotting the graphs of different countries
fig3 = make_subplots(rows = 2, cols = 2, specs = [[{"secondary_y":True}, {"secondary_y":True}], [{"secondary_y":True}, {"secondary_y":True}]], subplot_titles=("Bangladesh", "China", "Spain", "Italy"))

fig3.add_trace(go.Scatter(x = bd_cases['Date'], y = bd_cases['Cases'], name = "Cases", line = dict(color = 'yellow', width = 1), marker = dict(color = bd_cases['Cases'], coloraxis = "coloraxis")), 1, 1)
fig3.add_trace(go.Scatter(x = bd_deaths['Date'], y = bd_deaths['Deaths'], name = "Deaths", line = dict(color = 'red', width = 1), marker = dict(color = bd_deaths['Deaths'], coloraxis = "coloraxis")), 1, 1)
fig3.add_trace(go.Scatter(x = bd_recovered['Date'], y = bd_recovered['Recovered'], name = "Recovered", line = dict(color = 'green', width = 1), marker = dict(color = bd_recovered['Recovered'], coloraxis = "coloraxis")), 1, 1)

fig3.add_trace(go.Scatter(x = USA_cases['Date'], y = USA_cases['Cases'], name = "Cases", showlegend = False, line = dict(color = 'blue', width = 1), marker = dict(color = USA_cases['Cases'], coloraxis = "coloraxis")), 1, 2)
fig3.add_trace(go.Scatter(x = USA_deaths['Date'], y = USA_deaths['Deaths'], name = "Deaths", showlegend = False, line = dict(color = 'red', width = 1), marker = dict(color = USA_deaths['Deaths'], coloraxis = "coloraxis")), 1, 2)
fig3.add_trace(go.Scatter(x = USA_recovered['Date'], y = USA_recovered['Recovered'], name = "Recovered", showlegend = False, line = dict(color = 'green', width = 1), marker = dict(color = USA_recovered['Recovered'], coloraxis = "coloraxis")), 1, 2)

fig3.add_trace(go.Scatter(x = Spain_cases['Date'], y = Spain_cases['Cases'], name = "Cases", showlegend = False, line = dict(color = 'blue', width = 1), marker = dict(color = Spain_cases['Cases'], coloraxis = "coloraxis")), 2, 1)
fig3.add_trace(go.Scatter(x = Spain_deaths['Date'], y = Spain_deaths['Deaths'], name = "Deaths", showlegend = False, line = dict(color = 'red', width = 1), marker = dict(color = Spain_deaths['Deaths'], coloraxis = "coloraxis")), 2, 1)
fig3.add_trace(go.Scatter(x = Spain_recovered['Date'], y = Spain_recovered['Recovered'], name = "Recovered", showlegend = False, line = dict(color = 'green', width = 1), marker = dict(color = Spain_recovered['Recovered'], coloraxis = "coloraxis")), 2, 1)

fig3.add_trace(go.Scatter(x = Italy_cases['Date'], y = Italy_cases['Cases'], name = "Cases", showlegend = False, line = dict(color = 'blue', width = 1), marker = dict(color = Italy_cases['Cases'], coloraxis = "coloraxis")), 2, 2)
fig3.add_trace(go.Scatter(x = Italy_deaths['Date'], y = Italy_deaths['Deaths'], name = "Deaths", showlegend = False, line = dict(color = 'red', width = 1), marker = dict(color = Italy_deaths['Deaths'], coloraxis = "coloraxis")), 2, 2)
fig3.add_trace(go.Scatter(x = Italy_recovered['Date'], y = Italy_recovered['Recovered'], name = "Recovered", showlegend = False, line = dict(color = 'green', width = 1), marker = dict(color = Italy_recovered['Recovered'], coloraxis = "coloraxis")), 2, 2)

fig3.update_layout(coloraxis = dict(colorscale = 'Bluered_r'), showlegend = True, title_text = "Comparison Between Bangladesh and Other Three Countries", height = 600, width = 800)

fig3.update_yaxes(title_text = "Values", row = 1, col = 1)
fig3.update_yaxes(title_text = "Values", row = 1, col = 2)
fig3.update_yaxes(title_text = "Values", row = 2, col = 1)
fig3.update_yaxes(title_text = "Values", row = 2, col = 2)

fig3.show()

In [ ]:
data_set_copy = data_set_copy.nlargest(10,['currentConfirmedCount'])
print ("10 country with highest number of current confirmed count for Covid19")
bar_chart = data_set_copy.plot(x='countryEnglishName', y=['currentConfirmedCount','confirmedCount','curedCount','deadCount'],width=0.8,figsize=(10,8), align="center", kind="bar")

# to show value with corresponding value
for p in bar_chart.patches:
    bar_chart.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

plt.title("Graph for 10 country`s data set")
plt.legend()    
plt.show()

In [63]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')

summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary


#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

In [64]:
summary

{'updated': Timestamp('2020-04-21 00:00:00'),
 'since': Timestamp('2020-04-16 00:00:00'),
 'Cases': 2561043,
 'Deaths': 176983,
 'Cases (+)': 408606,
 'Deaths (+)': 33169,
 'China Cases': 83853,
 'China Deaths': 4636,
 'China Cases (+)': 450,
 'China Deaths (+)': 1290,
 'US Cases': 823786,
 'US Deaths': 44845,
 'US Cases (+)': 156194,
 'US Deaths (+)': 11915,
 'EU Cases': 1141410,
 'EU Deaths': 107909,
 'EU Cases (+)': 143342,
 'EU Deaths (+)': 15638}

In [65]:
#dataset of Coronavirus cases in Bangladesh
df = pd.read_csv("district_wise_cases_bangladesh.csv")
df

,District,Cases
0,Dhaka,1229
1,Gazipur,269
2,Kishoreganj,146
3,Madaripur,26
4,Manikganj,9
5,Narayanganj,469
6,Munshigonj,54
7,Narsingdi,136
8,Rajbari,10
9,Faridpur,5


In [66]:
df.style.background_gradient(cmap="Reds").hide_index()

District,Cases
Dhaka,1229
Gazipur,269
Kishoreganj,146
Madaripur,26
Manikganj,9
Narayanganj,469
Munshigonj,54
Narsingdi,136
Rajbari,10
Faridpur,5


In [69]:
total_cases = df["Cases"].sum()
print("Total Cases Found in Bangladesh until 21th April: ",total_cases)

Total Cases Found in Bangladesh until 21th April:  2769


In [70]:
%matplotlib inline

fig1 = go.Figure()

fig1.add_trace(go.Bar(x = df['District'], y = df['Cases'], marker = dict(color = df['Cases'], coloraxis = "coloraxis")))
fig1.update_layout(title = "Bar Chart showing District-wise Coronavirus Cases", xaxis_title = "Date", yaxis_title = "Cases", height = 600, width = 800)

fig1.show()

In [71]:
#dataset of districts' location in Bangladesh
dis_loc = pd.read_csv("district_coordinates_bangladesh.csv")
#dis_loc

In [72]:
#merging previous two dataset
df_updated = pd.merge(dis_loc, df, on = "District")
df_updated

,District,Latitude,Longitude,Cases
0,Dhaka,23.8103,90.4125,1229
1,Gazipur,23.9999,90.4203,269
2,Jamalpur,25.0831,89.7853,22
3,Kishoreganj,24.4260,90.9821,146
4,Madaripur,23.2393,90.1870,26
5,Manikganj,23.8617,90.0003,9
6,Narayanganj,23.6238,90.5000,469
7,Narsingdi,24.1344,90.7860,136
8,Rajbari,23.7639,89.6467,10
9,Tangail,24.3917,89.9948,13


In [73]:
#visualizing the data in map
map = folium.Map(location = [23.8,90], zoom_start = 7)
folium.TileLayer('stamenterrain').add_to(map)


for lat, long, value, name in zip(df_updated["Latitude"], df_updated["Longitude"], df_updated["Cases"], df_updated["District"]):
    folium.CircleMarker([lat, long], radius = value*0.15, color = 'crimson', fill = True, fill_color = 'crimson', popup = ("<b>District</b>: " + str(name) +"<br>""<b>Cases</b>: " + str(value)+ "<br>"), fill_opacity=0.3).add_to(map)

map
